In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-124571")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F2HGN84Z2 to authenticate.


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
cpu_cluster_name = "trainingcluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
      print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
# wait for completion
cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###

ps = param_sampling = RandomParameterSampling( {
        "--C": uniform(0.1, 1.0),
        "--max_iter": uniform(10, 100)     
    }
)

# Specify a Policy
#policy = ### YOUR CODE HERE ###
from azureml.train.hyperdrive import BanditPolicy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
est = SKLearn(source_directory="./training",
            compute_target=cpu_cluster,
             entry_script="train.py")


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=50,
                                max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

# visualise run detail using notebook widget
RunDetails(hyperdrive_run).show()

# wait for completion
hyperdrive_run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# get best run
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_hyperdrive_run.get_details()['runDefinition']['arguments'])

# list model files used for run
print(best_hyperdrive_run.get_file_names())

# register model for future deployment
os.makedirs('outputs', exist_ok=True)

hyperdrive_model = best_hyperdrive_run.register_model(model_name='hyperdriveModel', model_path='outputs/model.joblib')


In [ ]:
# view best model
print(run.get_metrics())

In [ ]:
# register model for later viewing or querying- mine
# register model
model = best_run.register_model(model_name='best_hyperdrive_model',
                           model_path='outputs/best_hyperdrive_model.joblib')
print(model.name, model.id, model.version, sep='\t')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url)


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)  ### YOUR DATA OBJECT HERE ###

# split into train and test sets
(x_train, y_train, x_test, y_test) = train_test_split(x, y, test_size= 0.3, random_state = 0)
label = 'y'

# merge x and y dataframes
train_data = pd.concat([x_train, y_train], axis=1)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='accuracy',
    training_data= train_data,
    label_column_name= label,
    n_cross_validations='2')

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

#from azureml.core.experiment import Experiment

#ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-logisticRegression'
#project_folder = './sample_projects/automl-logisticRegression'

experiment = Experiment(ws, experiment_name)

exp_run = experiment.submit(automl_config, show_output=True)

exp_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_model = run.get_metrics()

